<a href="https://colab.research.google.com/github/okana2ki/ML/blob/main/20231220_SwallowLLM_ipynb_%E3%81%AE%E3%82%B3%E3%83%94%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Demo for Swallow LLM by Tokyo Tech

公式サイト
https://tokyotech-llm.github.io/

モデル
https://huggingface.co/tokyotech-llm

★AICU media 「[東工大と産総研、英語の言語理解や対話で高い能力を持つ大規模言語モデル「Swallow」を公開 #SwallowLLM](https://note.com/aicu/n/n3eb8c1f2df02)」

★AICU media 「[東工大LLM「Swallow」を使ってGoogle Colabで遊んでみよう #SwallowLLM](https://note.com/aicu/n/nd0337d4952f3)」の解説コードです

参考：比較的初心者向けのGoogle Colabでの「Japanese Stable LM Gamma 7B」を動かす記事

■[Stability AI Japanが公開した30億パラメーターの日本語向けLLMを動かしてみた - 生成AIストリーム - 窓の杜](https://forest.watch.impress.co.jp/docs/serial/aistream/1544320.html)

Coded by Akihiko SHIRAI (kaitas[@o_ob](https://twitter.com/o_ob)) PoC開発や技術発信のお仕事歓迎です

# Step.1

[Google Colab Pro](https://colab.research.google.com/signup/pricing?hl=ja) を使ってGPUが利用できるインスタンスを作ります。具体的には「A100 GPU」もしくは「V100 GPU」以上を選びましょう。「A100」を選んでしばらく待てば、運が良ければ割り当てられます（利用できない時は「V100」になります）。



In [ ]:
# パッケージのインストール、いろいろあるけど Colab環境ならこれだけで動くはず
!pip install sentencepiece accelerate

In [ ]:
# @title Step.2 Tokenizer & Model Loading
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# @markdown [https://huggingface.co/tokyotech-llm](https://huggingface.co/tokyotech-llm) から利用したいモデルを選択してください。最初は 7b-instruct から始めるのがおすすめです。13bは ColabPro では動いています。70bはColabProでもダウンロードが難しいです。
tokenizer_model_name = "tokyotech-llm/Swallow-7b-instruct-hf" # @param ['tokyotech-llm/Swallow-7b-hf','tokyotech-llm/Swallow-7b-instruct-hf','tokyotech-llm/Swallow-13b-hf','tokyotech-llm/Swallow-13b-instruct-hf','tokyotech-llm/Swallow-70b-hf','tokyotech-llm/Swallow-70b-instruct-hf']
model_name = "tokyotech-llm/Swallow-7b-instruct-hf" # @param ['tokyotech-llm/Swallow-7b-hf','tokyotech-llm/Swallow-7b-instruct-hf','tokyotech-llm/Swallow-13b-hf','tokyotech-llm/Swallow-13b-instruct-hf','tokyotech-llm/Swallow-70b-hf','tokyotech-llm/Swallow-70b-instruct-hf']

# Load
tokenizer = AutoTokenizer.from_pretrained(tokenizer_model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, low_cpu_mem_usage=True ,device_map="auto")

# @title
PROMPT_DICT = {
    "prompt_input": (
        "以下に、あるタスクを説明する指示があり、それに付随する入力が更なる文脈を提供しています。"
        "リクエストを適切に完了するための回答を記述してください。\n\n"
        "### 指示:\n{instruction}\n\n### 入力:\n{input}\n\n### 応答:"

    ),
    "prompt_no_input": (
        "以下に、あるタスクを説明する指示があります。"
        "リクエストを適切に完了するための回答を記述してください。\n\n"
        "### 指示:\n{instruction}\n\n### 応答:"
    ),
}

def create_prompt(instruction, input=None):
    """
    Generates a prompt based on the given instruction and an optional input.
    If input is provided, it uses the 'prompt_input' template from PROMPT_DICT.
    If no input is provided, it uses the 'prompt_no_input' template.

    Args:
        instruction (str): The instruction describing the task.
        input (str, optional): Additional input providing context for the task. Default is None.

    Returns:
        str: The generated prompt.
    """
    if input:
        # Use the 'prompt_input' template when additional input is provided
        return PROMPT_DICT["prompt_input"].format(instruction=instruction, input=input)
    else:
        # Use the 'prompt_no_input' template when no additional input is provided
        return PROMPT_DICT["prompt_no_input"].format(instruction=instruction)


In [18]:
# @title Step.3 Settings & Prompts
instruction_example = "以下のトピックに関する詳細な情報を提供してください。" # @param {type: "string"}
input_example = "Oka Natsuki" # @param {type: "string"}
Do_sample=True #@param {type:"boolean"}

if Do_sample:
  temperature = 1 #@param {type:"slider", min:0, max:2, step:0.1}
  top_p = 0.95 #@param {type:"slider", min:0, max:1, step:0.01}

max_new_tokens=512 #@param {type:"slider", min:128, max:1024, step:64}

# Example usage
# instruction_example = "以下のトピックに関する詳細な情報を提供してください。"
# input_example = "東京工業大学の主なキャンパスについて教えてください"
prompt = create_prompt(instruction_example, input_example)

input_ids = tokenizer.encode(
    prompt,
    add_special_tokens=False,
    return_tensors="pt"
)

tokens = model.generate(
    input_ids.to(device=model.device),
    max_new_tokens=max_new_tokens,
    temperature=temperature,
    top_p=top_p,
    do_sample=Do_sample,
)

out = tokenizer.decode(tokens[0], skip_special_tokens=True)
print(out)


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.53` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


以下に、あるタスクを説明する指示があり、それに付随する入力が更なる文脈を提供しています。リクエストを適切に完了するための回答を記述してください。

### 指示:
以下のトピックに関する詳細な情報を提供してください。

### 入力:
Oka Natsuki

### 応答:Oka Natsukiは、日本の女優、モデル、歌手です。彼女は2011年にテレビシリーズ「Gakuen Alice」でデビューし、2012年に映画「Kimi ga Suki dake」で主演しました。彼女はまた、2013年の映画「The Last Letter」で主演し、2014年の映画「The Last Letter」で主演しました。彼女はまた、2015年の映画「The Last Letter」で主演しました。彼女はまた、2016年の映画「The Last Letter」で主演しました。彼女はまた、2017年の映画「The Last Letter」で主演しました。彼女はまた、2018年の映画「The Last Letter」で主演しました。彼女はまた、2019年の映画「The Last Letter」で主演しました。彼女はまた、2020年の映画「The Last Letter」で主演しました。彼女はまた、2021年の映画「The Last Letter」で主演しました。彼女はまた、2022年の映画「The Last Letter」で主演しました。彼女はまた、2023年の映画「The Last Letter」で主演しました。
